In [15]:
import pandas as pd
import duckdb
import constants

import os
from dagster._utils.backoff import backoff

Some of the assets are materialized in a DuckDB database, so let's set up a connection:

In [16]:
duckdb_path = '/workspaces/project-dagster-university/dagster_university/dagster_essentials/data/staging/data.duckdb'

conn = backoff(
        fn=duckdb.connect,
        max_retries=10,
        retry_on=(RuntimeError, duckdb.IOException),
        kwargs={"database": duckdb_path},
    )

In [9]:

# Fomr trips.py
query_z = """
          SELECT *
          FROM zones
          LIMIT 10
          """
zones = conn.query(query_z).fetchdf()

query_t = """
          SELECT *
          FROM trips
          LIMIT 10
          """
trips = conn.query(query_t).fetchdf()


# From metrics.py
trips_filepath = '/workspaces/project-dagster-university/dagster_university/dagster_essentials/data/outputs/trips_by_week.csv'
# trips_by_week = pd.read_csv(constants.TRIPS_BY_WEEK_FILE_PATH)
trips_by_week = pd.read_csv(trips_filepath)


In [11]:
print(zones)

   zone_id                     zone        borough  \
0        1           Newark Airport            EWR   
1        2              Jamaica Bay         Queens   
2        3  Allerton/Pelham Gardens          Bronx   
3        4            Alphabet City      Manhattan   
4        5            Arden Heights  Staten Island   
5        6  Arrochar/Fort Wadsworth  Staten Island   
6        7                  Astoria         Queens   
7        8             Astoria Park         Queens   
8        9               Auburndale         Queens   
9       24             Bloomingdale      Manhattan   

                                            geometry  
0  MULTIPOLYGON (((-74.18445299999996 40.69499599...  
1  MULTIPOLYGON (((-73.82337597260663 40.63898704...  
2  MULTIPOLYGON (((-73.84792614099985 40.87134223...  
3  MULTIPOLYGON (((-73.97177410965318 40.72582128...  
4  MULTIPOLYGON (((-74.17421738099989 40.56256808...  
5  MULTIPOLYGON (((-74.06367318899999 40.60219816...  
6  MULTIPOLYGON (((-

In [12]:
print(trips)

   vendor_id  pickup_zone_id  dropoff_zone_id  rate_code_id  payment_type  \
0          2             238               42             1             2   
1          2             138              231             1             1   
2          1             140              186             1             1   
3          1             140               43             1             1   
4          2              79              137             1             1   
5          1             162              137             1             1   
6          1             170               48             1             1   
7          1              48              164             1             2   
8          1             113               61             1             1   
9          2             239              263             1             1   

     dropoff_datetime     pickup_datetime  trip_distance  passenger_count  \
0 2023-03-01 00:16:43 2023-03-01 00:06:43           0.00                1  

In [13]:
print(trips_by_week)

    Unnamed: 0      period  num_trips  passenger_count  total_amount  \
0            0  2023-03-06     774813        1019293.0  2.119257e+07   
1            1  2023-03-13     787016        1044648.0  2.209606e+07   
2            2  2023-02-27     560895         739953.0  1.518185e+07   
3            3  2023-02-20         55             54.0  9.109000e+02   
4            4  2002-12-30          5              7.0  3.267000e+02   
5            5  2001-01-01          3              5.0  1.717000e+02   
6            6  2023-03-20     734288         969859.0  2.067543e+07   
7            7  2008-12-29          2              2.0  6.595000e+01   
8            8  2023-03-27     546619         707576.0  1.548723e+07   
9            9  2014-11-17          1              2.0  5.238000e+01   
10          10  2023-04-03         69            100.0  1.693550e+03   

    trip_distance  
0      2739557.76  
1      3079735.47  
2      2412606.42  
3            0.01  
4           62.67  
5           34.

aggregating trips_by_week

In [ ]:
query = """
            SELECT *
            FROM trips
            WHERE pickup_datetime >= '2023-01-01' AND pickup_datetime < '2023-04-01'
            """

In [22]:
query = """
            SELECT
                DATE_TRUNC('week', pickup_datetime) as period,
                COUNT(*) as num_trips,
                SUM(passenger_count) as passenger_count,
                SUM(total_amount) as total_amount,
                SUM(trip_distance) as trip_distance,
            FROM trips
            WHERE pickup_datetime >= '2023-01-01' AND pickup_datetime < '2023-04-01'
            GROUP BY DATE_TRUNC('week', pickup_datetime)
            ORDER BY period  DESC
            """

In [23]:
trips_by_week = conn.execute(query).fetch_df()

In [24]:
print(trips_by_week)

      period  num_trips  passenger_count  total_amount  trip_distance
0 2023-03-27     546603         707539.0  1.548673e+07     2119944.43
1 2023-03-20     734288         969859.0  2.067543e+07     2935650.00
2 2023-03-13     787016        1044648.0  2.209606e+07     3079735.47
3 2023-03-06     774813        1019293.0  2.119257e+07     2739557.76
4 2023-02-27     560895         739953.0  1.518185e+07     2412606.42
5 2023-02-20         55             54.0  9.109000e+02           0.01
